# 1º teste para o AWS Textract

Uso direto da API do SDK da Amazon com a linguagem Python

Pré-procedimento:
- Baixar o AWS CLI ()
- Baixar o SDK Python da AWS (Boto3)
- Configurar com o comando de linha "aws configure". *Procedimento detalhado nas anotações sobre o Textract
- Configurar um balde simples (S3) de informações nos serviços da AWS, para enviar/puxar os documentos
- Baixar o pacote da AWS com o pip ("!pip install botocore" e "!pip install boto3")
  * Sempre instalar o core primeiro, e com a exclamação de persistência no começo, senão dá problema *
 
### Fonte: <https://www.youtube.com/watch?v=-SpHPW3RTx8>

In [34]:
%pip install botocore

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [35]:
%pip install boto3

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.




[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
%pip install textract-trp

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:
import boto3
# Biblioteca para a manipulação de Documentos processados pelo Texract
from trp import Document

# Salvar o nome do balde que será usado para guardar as imagens
# E das imagems de teste usada
nomeBalde = "buck-textract"
nomeImagem = [
    'teste01-wallpaper.jpg',
    'teste02-formulario.png',
    'teste03-tabela.pdf'
]

# Iniciar a seção com o usuário remoto
secao = boto3.session.Session()

# Importar a biblioteca dos serviços usados (Textract, Balde S3)
s3 = secao.client("s3")
textract = secao.client("textract")

In [38]:
# Teste da API AWS: Requisitar uma imagem de teste de dentro do balde
reqImagem = s3.get_object(Bucket=nomeBalde, Key=nomeImagem[1])
print(reqImagem)

{'ResponseMetadata': {'RequestId': 'W7VF2AEQAP0YVX77', 'HostId': 'flhlm2fjvHF9mkoYS7ypBmp2+97TG3t1MhO7nPC4o9j54Q9XEVYFglCV8ezfGpe4j8wDeJHG+0LB5qhiffJuaLSBcP31rHY1+5SC93+c8Vs=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'flhlm2fjvHF9mkoYS7ypBmp2+97TG3t1MhO7nPC4o9j54Q9XEVYFglCV8ezfGpe4j8wDeJHG+0LB5qhiffJuaLSBcP31rHY1+5SC93+c8Vs=', 'x-amz-request-id': 'W7VF2AEQAP0YVX77', 'date': 'Fri, 28 Feb 2025 14:34:17 GMT', 'last-modified': 'Wed, 22 Jan 2025 20:02:51 GMT', 'etag': '"9c6b7ec23bf7bc479c172f7ca1296f5b"', 'x-amz-checksum-crc64nvme': 'Lz1tNQHo++U=', 'x-amz-checksum-type': 'FULL_OBJECT', 'x-amz-server-side-encryption': 'AES256', 'accept-ranges': 'bytes', 'content-type': 'image/png', 'content-length': '118637', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'AcceptRanges': 'bytes', 'LastModified': datetime.datetime(2025, 1, 22, 20, 2, 51, tzinfo=tzutc()), 'ContentLength': 118637, 'ETag': '"9c6b7ec23bf7bc479c172f7ca1296f5b"', 'ChecksumCRC64NVME': 'Lz1tNQHo++U=', 'ChecksumType': 'FULL_

In [39]:
# Com a confirmação de que a imagem está lá, fazer a leitura da imagem com o Textract
# 1ª imagem: Wallpaper com as frases:
"""
    - SALT - ASBESTOS - CURTAIN
    - APERTURE
    - SCIENCE INNOVATORS
"""
# OBS: Função 'detect_document_text' só lê as linhas de texto puras do documento
# Função geral para ler textos no modo padrão
def reqPadrao(balde,imagem):
    reqDigImagem0 = textract.detect_document_text(
        # Informar o tipo de objeto que contém o documento esperado
        Document = {
            'S3Object': {
                'Bucket': balde,
                'Name': imagem
            }
        }
    )
    print('Informações reconhecidas da imagem "{}":'.format(imagem))
    
    # Iterar as páginas do arquivo
    linha = 0
    for bloco in reqDigImagem0['Blocks']:
        if bloco["BlockType"] == "LINE":
            print("Linha {}: {}".format(linha, bloco["Text"]))
            linha = linha +1

# Chamar a função para a 1a imagem
reqPadrao(nomeBalde, nomeImagem[0])

Informações reconhecidas da imagem "teste01-wallpaper.jpg":
Linha 0: ASBESTOS
Linha 1: APERTURE
Linha 2: SCIENCE INNOVATORS


RESULTADO: Faltam algumas palavras, elas estão curvadas e de lado

In [40]:
# 2ª imagem: screenshot do formulário de criação de usuário IAM da AWS, com os inputs:
"""
    - Nome do usuário
    - Senha (radio button)
    - Endereço de email
    - Confirmar Endereço de email
    - Nome
"""
# OBS: a função 'analyze_document' vai pegar as informações do documento de acordo com um modelo
reqDigImagem1 = textract.analyze_document(
    # Informar o tipo de objeto que contém o documento esperado
    Document = {
        'S3Object': {
            'Bucket': nomeBalde,
            'Name': nomeImagem[1]
        }
    },
    # Informar qual o tipo de documento vai ser analisado, no caso, um formulário
    FeatureTypes = ["FORMS"]
)
doc = Document(reqDigImagem1)
print('Informações reconhecidas da imagem "{}":'.format(nomeImagem[1]))

# Iterar as páginas do arquivo
for pagina in doc.pages:
    for campo in pagina.form.fields:
        print("Chave '{}' - Valor '{}'".format(campo.key, campo.value))

Informações reconhecidas da imagem "teste02-formulario.png":
Chave 'Envie um e-mail para O usuário com as instruções para configuração da senha.' - Valor 'SELECTED'
Chave 'Nome' - Valor 'Insira o nome'
Chave 'Confirmar endereço de e-mail' - Valor 'email@example.com'
Chave 'Endereço de e-mail' - Valor 'email@example.com'
Chave 'Gere uma senha única que possa ser compartilhada com este usuário.' - Valor 'NOT_SELECTED'
Chave 'Nome do usuário' - Valor 'Esse nome de usuário será necessário para que esse usuário faça login no AWS access portal. O nome de usuário não pode ser alterado posteriormente. Insira O nome de usuário'


RESULTADO: Todas as informções condizem com o documento original

In [41]:
# 3ª imagem: um documento em pdf com todas as grandezas de tamanho (de 10e-24 a 10e24)
reqDigImagem2 = textract.analyze_document(
    # Informar o tipo de objeto que contém o documento esperado
    Document = {
        'S3Object': {
            'Bucket': nomeBalde,
            'Name': nomeImagem[2]
        }
    },
    # Informar qual o tipo de documento vai ser analisado, no caso, um formulário
    FeatureTypes = ["TABLES"]
)
doc = Document(reqDigImagem2)
print('Informações reconhecidas da imagem "{}":'.format(nomeImagem[2]))

# Iterar as páginas do arquivo
nTabela = 0
for pagina in doc.pages:
    for tabela in pagina.tables:
        print("Tabela {}:".format(nTabela))
        nTabela = nTabela +1
        for y, linha in enumerate(tabela.rows):
            for x, celula in enumerate(linha.cells):
                print("\t[{}][{}] = {}".format(x, y, celula.text))
            print()

Informações reconhecidas da imagem "teste03-tabela.pdf":
Tabela 0:
	[0][0] = NOME 
	[1][0] = SIMBOLO 
	[2][0] = POTENCIA 
	[3][0] = NUMERO 

	[0][1] = yotta 
	[1][1] = Y 
	[2][1] = 1024 
	[3][1] = 1 000 000 000 000 000 000 000 000 

	[0][2] = zetta 
	[1][2] = Z 
	[2][2] = 1021 
	[3][2] = 1000 000 000 000 000 000 000 

	[0][3] = exa 
	[1][3] = E 
	[2][3] = 1018 
	[3][3] = 1000 000 000 000 000 000 

	[0][4] = peta 
	[1][4] = P 
	[2][4] = 1015 
	[3][4] = 1 000 000000 000 000 

	[0][5] = tera 
	[1][5] = T 
	[2][5] = 1012 
	[3][5] = 000 000 000 000 

	[0][6] = giga 
	[1][6] = G 
	[2][6] = 10° 
	[3][6] = 1000 000 000 

	[0][7] = mega 
	[1][7] = M 
	[2][7] = 106 
	[3][7] = 1000000 

	[0][8] = quilo 
	[1][8] = K 
	[2][8] = 103 
	[3][8] = 1000 

	[0][9] = hecto 
	[1][9] = H 
	[2][9] = 102 
	[3][9] = 100 

	[0][10] = deca 
	[1][10] = Da 
	[2][10] = 101 
	[3][10] = 10 

	[0][11] = deci 
	[1][11] = d 
	[2][11] = 10-1 
	[3][11] = 0,1 

	[0][12] = centi 
	[1][12] = C 
	[2][12] = 10-2 
	[3][12] = 0,0

RESULTADO: com excessão dos números escritos em potência de 10, as informações condizem com o documento original

# 2º Teste para o AWS Textract

Será levada em conta agora a capacidade que a ferramenta tem de identificar o texto, ou seja, serão usadas imagens de baixa qualidade (borradas, manuscritas, tortas, etc) para testar até onde o Textract consegue decifrar um texto.

Para esse teste, foram adicionadas 16 novas imagens ao balde na plataforma AWS, conforme o padrão:
- Texto manuscrito em letra de forma;
- Mesmo texto em letra de forma, mas com a foto tremida;
- Mesmo texto em letra de forma, mas com a foto torta;
- Mesmo texto em letra de forma, mas com a foto tremida e torta;

Os 4 tipos de imagem foram replicados para as caligrafias de forma em lápis, cursiva em lápis, de forma em caneta, cursiva em caneta, respectivamente.

Todas as imagens possuem uma mesma frase aleatória para padronização: "Janeiro tem 31 dias. Fevereiro tem 28 noites. Março tem 5 domingos. E Abril tem 4 segundas."

In [42]:
# Atualizar o conjunto de imagens
novaLista = [
    'teste04-forma.jpg',
    'teste05-forma-tremida.jpg',
    'teste06-forma-torta.jpg',
    'teste07-forma-tremida-torta.jpg',
    'teste08-cursiva.jpg',
    'teste09-cursiva-tremida.jpg',
    'teste10-cursiva-torta.jpg',
    'teste11-cursiva-tremida-torta.jpg',
    'teste12-forma-caneta.jpg',
    'teste13-forma-caneta-tremida.jpg',
    'teste14-forma-caneta-torta.jpg',
    'teste15-forma-caneta-tremida-torta.jpg',
    'teste16-cursiva-caneta.jpg',
    'teste17-cursiva-caneta-tremida.jpg',
    'teste18-cursiva-caneta-torta.jpg',
    'teste19-cursiva-caneta-tremida-torta.jpg',
]
for item in novaLista: nomeImagem.append(item)
print(nomeImagem)

['teste01-wallpaper.jpg', 'teste02-formulario.png', 'teste03-tabela.pdf', 'teste04-forma.jpg', 'teste05-forma-tremida.jpg', 'teste06-forma-torta.jpg', 'teste07-forma-tremida-torta.jpg', 'teste08-cursiva.jpg', 'teste09-cursiva-tremida.jpg', 'teste10-cursiva-torta.jpg', 'teste11-cursiva-tremida-torta.jpg', 'teste12-forma-caneta.jpg', 'teste13-forma-caneta-tremida.jpg', 'teste14-forma-caneta-torta.jpg', 'teste15-forma-caneta-tremida-torta.jpg', 'teste16-cursiva-caneta.jpg', 'teste17-cursiva-caneta-tremida.jpg', 'teste18-cursiva-caneta-torta.jpg', 'teste19-cursiva-caneta-tremida-torta.jpg']


In [43]:
# Imagem 4 - Letra de forma, boa qualidade de imagem
reqPadrao(nomeBalde, nomeImagem[3])

Informações reconhecidas da imagem "teste04-forma.jpg":
Linha 0: JANEiRO TEM 31
Linha 1: DiAS. I FEVEREIRO
Linha 2: TEM 28 NOTTES.
Linha 3: MARCO TEM 5
Linha 4: DOMINGOS. E ABRiL
Linha 5: possui 4 SE GUNDA.


RESULTADO: por conta da caligrafia, parte dos caracteres

In [ ]:
# Imagem 5 - Letra de forma, imagem tremida
reqPadrao(nomeBalde, nomeImagem[4])

Informações reconhecidas da imagem "teste06-forma-torta.jpg":
Linha 0: JANEiRO TEM 31
Linha 1: DiAS. FEVEREiRO
Linha 2: TEM 28 NOiTES.
Linha 3: MARCO TEM 5
Linha 4: DOMINGOS. E ABRiL
Linha 5: Possui 4 SE GUNDA.


RESULTADO: a qualidade da imagem atrapalhou bastante a identificação dos caractéres, fazendo perder alguns deles e gerando uma nova linha que não existe.

In [ ]:
# Imagem 6 - Letra de forma, imagem torta
reqPadrao(nomeBalde, nomeImagem[5])

Informações reconhecidas da imagem "teste07-forma-tremida-torta.jpg":
Linha 0: F8


RESULTADO: mesmo resultado da imagem 4, tirar a foto de maneira torta não teve nenhum efeito.

In [ ]:
# Imagem 7 - Letra de forma, imagem tremida e torta
reqPadrao(nomeBalde, nomeImagem[6])

Informações reconhecidas da imagem "teste08-cursiva.jpg":
Linha 0: Laneira tem 31 diar.
Linha 1: Fevereire tem 28 moi-
Linha 2: ter. Marga tem 5
Linha 3: domingos. E Abril
Linha 4: possui 4 regundas.


RESULTADO: a página torta com a foto tremida tornou impossível ler qualquer coisa

In [ ]:
# Imagem 8 - Letra de cursiva, boa qualidade de imagem
reqPadrao(nomeBalde, nomeImagem[7])

Informações reconhecidas da imagem "teste05-forma-tremida.jpg":
Linha 0: JANEIRO TEA
Linha 1: DiAS. FEVEREING
Linha 2: TEM 28 NO:TES,
Linha 3: MARCO TEM 5
Linha 4: DOMINGOS. E ABR
Linha 5: POSSUI 4 SE GUNDA
Linha 6: S


RESULTADO: a interferência da caligrafia foi maior ainda, a mensagem deixou de ter qualquer sentido. Os caractéres maiúsculos e minusculos das frases foram diferenciados.

In [49]:
# Imagem 9 - Letra de cursiva, imagem tremida
reqPadrao(nomeBalde, nomeImagem[8])

Informações reconhecidas da imagem "teste09-cursiva-tremida.jpg":
Linha 0: danline tem 31 dias.
Linha 1: Invoice tem 28 moi-
Linha 2: the Marca tem 5
Linha 3: domingos. E Abril
Linha 4: possion 4 regundar
Linha 5: X


RESULTADO: a combinação da foto tremida com a caligrafia cursiva piorou a detecção da maioria dos carácteres, porém algumas poucas palavras ainda puderam ser identificadas.

In [50]:
# Imagem 10 - Letra de cursiva, imagem torta
reqPadrao(nomeBalde, nomeImagem[9])

Informações reconhecidas da imagem "teste10-cursiva-torta.jpg":
Linha 0: F6
Linha 1: F7
Linha 2: F8


RESULTADO: As palavras na letra cursiva não puderam ser reconhecidas, então o algoritmo tentou deduzir erroneamente alguma informação dos símbolos.

In [51]:
# Imagem 11 - Letra de cursiva, imagem tremida e torta
reqPadrao(nomeBalde, nomeImagem[10])

Informações reconhecidas da imagem "teste11-cursiva-tremida-torta.jpg":


RESULTADO: o algoritmo não teve a menor noção sobre nenhum dos símbolos escritos.

In [52]:
# Imagem 12 - Letra de forma, boa qualidade de imagem
reqPadrao(nomeBalde, nomeImagem[11])

Informações reconhecidas da imagem "teste12-forma-caneta.jpg":
Linha 0: JANEIRO TEM
Linha 1: 31 DIAS. FEVEREI
Linha 2: RO TEM 28 NOI-
Linha 3: TES. MARCO TEM
Linha 4: 5 DOMINGOS. E ABRiL
Linha 5: POSSUi 4 SEGUNDAS.


RESULTADO: resultados mais claros que a caligrafia no papel, única letra confundida foi o 'ç'.

In [53]:
# Imagem 13 - Letra de forma, de caneta, imagem tremida
reqPadrao(nomeBalde, nomeImagem[12])

Informações reconhecidas da imagem "teste13-forma-caneta-tremida.jpg":
Linha 0: JANEIRO TEM
Linha 1: 31 DIAS. FEVEREI-
Linha 2: RO TEM 28 NOI-
Linha 3: TES. MARÇO TEM
Linha 4: 5 DOMINGOS. EABRIL
Linha 5: POSSUi 4 SEGUNDAS


RESULTADO: mesmo resultado que a foto normal.

In [54]:
# Imagem 14 - Letra de forma, de caneta, imagem torta
reqPadrao(nomeBalde, nomeImagem[13])

Informações reconhecidas da imagem "teste14-forma-caneta-torta.jpg":


RESULTADO: nenhum caractér pode ser reconhecido.

In [55]:
# Imagem 15 - Letra de forma, de caneta, imagem tremida e torta
reqPadrao(nomeBalde, nomeImagem[14])

Informações reconhecidas da imagem "teste15-forma-caneta-tremida-torta.jpg":


RESULTADO: nenhum caractér pode ser reconhecido.

In [56]:
# Imagem 16 - Letra de cursiva, de caneta, boa qualidade de imagem
reqPadrao(nomeBalde, nomeImagem[15])

Informações reconhecidas da imagem "teste16-cursiva-caneta.jpg":
Linha 0: " Laneira tem 31 dias.
Linha 1: Ferrreira tem 28 noites.
Linha 2: Marca tem 5 domingos.
Linha 3: E abril porrui 4 regun-
Linha 4: das.


RESULTADO: mesmo resultado que a caligradia em lápis, tirando as aspas no início, que não existem no original.

In [57]:
# Imagem 17 - Letra de cursiva, de caneta, imagem tremida
reqPadrao(nomeBalde, nomeImagem[16])

Informações reconhecidas da imagem "teste17-cursiva-caneta-tremida.jpg":
Linha 0: Janeira tem 31 dias.
Linha 1: Ferrreira tem 28 noites.
Linha 2: Marca tem 5 domingor
Linha 3: E abril porrui 4 regun
Linha 4: dar.


RESULTADO: sem muitas alterações em relação ao lápis

In [58]:
# Imagem 18 - Letra de cursiva, de caneta, imagem torta
reqPadrao(nomeBalde, nomeImagem[17])

Informações reconhecidas da imagem "teste18-cursiva-caneta-torta.jpg":


RESULTADO: nenhum caractér pode ser reconhecido.

In [59]:
# Imagem 19 - Letra de cursiva, de caneta, imagem tremida e torta
reqPadrao(nomeBalde, nomeImagem[18])

Informações reconhecidas da imagem "teste19-cursiva-caneta-tremida-torta.jpg":
Linha 0: Laneira tem 31 dias.
Linha 1: Marga tem 5 damingon.
Linha 2: Everura tem 28 smailer noites,
Linha 3: des. Edvil possui Iregun


RESULTADO: Ao contrário do esperado, a foto com mais distorções funcionou da mesma maneira que a original.